# Bildverarbeitung mit Python
Gert-Ludwig Ingold
<div style="margin-top:10ex;font-size:smaller">Quellen: `git clone https://github.com/gertingold/lit2017`</div>

# Bilder sind auch nur Daten
![foo](img/rgbarray.png)
![foo](img/greyarray.png)

Numpy ndarray

`scikit-image`

* Webseite: http://scikit-image.org
* Quellen: https://github.com/scikit-image/scikit-image


In [4]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import skimage